In [ ]:
%pip install numpy pandas nltk
import nltk
nltk.download('stopwords')

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
data = pd.read_csv('../dataset/dataset.csv', sep='|')
data.Reviews=data.Reviews.astype(str)

#Transform text to lowercase
data['Reviews'] = data['Reviews'].apply(lambda x: x.lower()) 
#Removing all punctuations and special characters
data['Reviews'] = data['Reviews'].apply(lambda x: re.sub('[,.]', '', x))

In [ ]:
stopwords_list = stopwords.words('english')
data['Reviews'] = data['Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

In [ ]:
print(data)

In [ ]:
from textblob import TextBlob

In [ ]:
score_list=[]

for text in data['Reviews']:
  
  sentence = TextBlob(text)
  
  score = sentence.sentiment.polarity
  score_list.append(score)

In [ ]:
import statistics
median = statistics.median(score_list)
print(median)

In [ ]:
sentiment_label = []
for score in score_list:
  if score > median:
    sentiment_label.append('positive')
  elif score < median:
    sentiment_label.append('negative')
  else:
    sentiment_label.append('neutral')
print(sentiment_label)

In [ ]:
dataset = data
dataset['SCORE']=score_list
dataset['SENTIMENT_LABEL']=sentiment_label
print(dataset)
dataset.to_csv("../dataset/TextBlobResultswithMedian.csv", sep='|')

In [ ]:
human_data = pd.read_csv('../dataset/HumanAnnotatedDataset.csv', sep=',')

Fal_pos = 0.0
Fal_neg = 0.0
Tru_pos = 0.0
Tru_neg = 0.0


for ind in range(len(sentiment_label)):
  
    if sentiment_label[ind] == 'positive' and human_data['HUMAN_LABEL'][ind] == 'negative':
        Fal_pos += 1
    elif sentiment_label[ind] == 'negative' and human_data['HUMAN_LABEL'][ind] == 'positive':
        Fal_neg += 1
    elif sentiment_label[ind] == 'positive' and human_data['HUMAN_LABEL'][ind] == 'positive':
        Tru_pos += 1
    elif sentiment_label[ind] == 'negative' and human_data['HUMAN_LABEL'][ind] == 'negative':
        Tru_neg += 1

Accur = (Tru_pos + Tru_neg)/(Tru_pos + Tru_neg + Fal_pos + Fal_neg)
Prec = (Tru_pos)/(Tru_pos + Fal_pos)
Recal = (Tru_pos)/(Tru_pos + Fal_neg)
FScore = (2*Prec*Recal)/(Prec + Recal)
        
    
print("Accuracy: ", Accur)
print("Precision: ", Prec)
print("Recall: ", Recal)
print("F-Score: ", FScore)